#Dependencies and Environmet Set-Up


In [ ]:
!pip install langchain
!pip install openai
!pip install azure-storage-blob
!pip install tiktoken


In [ ]:
import os
from langchain.retrievers import AzureCognitiveSearchRetriever
from langchain.embeddings import OpenAIEmbeddings
from langchain.vectorstores import Chroma
from langchain.llms import AzureOpenAI
from langchain.chains import RetrievalQA

# ".env" FILE

In [ ]:
os.environ["AZURE_COGNITIVE_SEARCH_SERVICE_NAME"] = "acs-openai-test-3"
os.environ["AZURE_COGNITIVE_SEARCH_INDEX_NAME"] = "azureblob-index-17files"
os.environ["AZURE_COGNITIVE_SEARCH_API_KEY"] = ""

In [ ]:
import os

os.environ["OPENAI_API_TYPE"] = "azure"
os.environ["OPENAI_API_VERSION"] = "2023-03-15-preview"
os.environ["OPENAI_API_BASE"] = "https://testnew.openai.azure.com/"
os.environ["OPENAI_API_KEY"] = ""

# Retrival Chain

In [ ]:
retriever = AzureCognitiveSearchRetriever(content_key="content")

In [ ]:
query = input("Enter your query: ")

Enter your query: Dylan salary


In [ ]:
docs = retriever.get_relevant_documents(query)
docs

# Extracting relevant content


In [ ]:
#document= Document(page_content)
page_content = docs[0].page_content
print(page_content)


In [ ]:
os.environ["OPENAI_API_KEY"] = ""

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import AzureChatOpenAI
qa = RetrievalQA.from_chain_type(llm=AzureChatOpenAI(deployment_name="testgptturbo",model_name="gpt-35-turbo-16k", temperature=0, max_tokens=3000), chain_type="stuff", retriever=retriever)
qa.run(query)

ValidationError: ignored

In [ ]:
from langchain.llms import OpenAI
from langchain.chat_models import ChatOpenAI
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0, max_tokens=3000), chain_type="stuff", retriever=retriever)
qa.run(query)

InvalidRequestError: ignored

Working convrsation type chatbot with buffer mmemory
#### **STEP 1**

In [ ]:
from langchain.chains.question_answering import load_qa_chain
from langchain.llms import OpenAI
from langchain.prompts import PromptTemplate
from langchain.memory import ConversationBufferMemory

template = """You are a chatbot having a conversation with a human.

Given the following extracted parts of a long document and a question, create a final answer.

{context}

{chat_history}
Human: {human_input}
Chatbot:"""

prompt = PromptTemplate(
    input_variables=["chat_history", "human_input", "context"], template=template
)

# chat completion llm
llm = ChatOpenAI(
    openai_api_key= "",
    model_name='gpt-3.5-turbo',
    temperature=0.0
)
# conversational memory
memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    input_key="human_input"
)
# retrieval qa chain
chain =load_qa_chain(llm=ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0, max_tokens=3000),chain_type="stuff", memory=memory, prompt=prompt)



In [ ]:

chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': "Dylan Choi's annual salary is $231,141."}

In [ ]:
print(chain.memory.buffer)

[HumanMessage(content='Dylan Salary', additional_kwargs={}, example=False), AIMessage(content="Dylan Choi's annual salary is $231,141.", additional_kwargs={}, example=False)]


In [ ]:
query = "What is his emp code? "
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': "Dylan Choi's employee code is E00304."}

In [ ]:
query = "What is his department? "
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': "Dylan Choi's department is IT."}

In [ ]:
query = "Name of employee withe previous emp code? "
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': 'The previous employee with the employee code E00304 is Carson Lu.'}

In [ ]:
query = "carson lu emp code? "
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': "Carson Lu's employee code is E03343."}

In [ ]:
query = "employee in same department as him "
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': 'The employees in the same department as Dylan Choi (IT) are Emily Davis, Theodore Dinh, Penelope Jordan, David Barnes, and Nora Brown.'}

In [ ]:
query = "Carson lu emp dempt? "
chain({"input_documents": docs, "human_input": query}, return_only_outputs=True)

{'output_text': "Carson Lu's department is Engineering."}

### **STEP 2**
memory with databasde and agent

### **STEP 3**
memory with databasde and agent

In [ ]:
from langchain.agents import ZeroShotAgent, AgentExecutor
from langchain.memory import ConversationBufferMemory
from langchain.memory.chat_memory import ChatMessageHistory
from langchain.memory.chat_message_histories import RedisChatMessageHistory
from langchain import OpenAI, LLMChain


In [ ]:
from langchain.agents import Tool, initialize_agent
from langchain import LLMMathChain
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'use this tool when answering general knowledge queries to get '
            'more information about the topic'
        )
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

In [ ]:
prefix = """Have a conversation with a human, answering the following questions as best you can. You have access to the following tools:"""
suffix = """Begin!"

{chat_history}
Question: {input}
{agent_scratchpad}"""

prompt = ZeroShotAgent.create_prompt(
    tools,
    prefix=prefix,
    suffix=suffix,
    input_variables=["input", "chat_history", "agent_scratchpad"],
)

In [ ]:
message_history = RedisChatMessageHistory(
    url="redis://localhost:6379/0", ttl=600, session_id="my-session"
)

memory = ConversationBufferMemory(
    memory_key="chat_history", k=5, chat_memory=message_history
)

In [ ]:
llm_chain = LLMChain(llm=OpenAI(temperature=0), prompt=prompt)
agent = ZeroShotAgent(llm_chain=llm_chain, tools=tools, verbose=True)
agent_chain = AgentExecutor.from_agent_and_tools(
    agent=agent, tools=tools, verbose=True, memory=memory
)

In [ ]:
agent_chain.run(input="Salary of Dylan?")

In [ ]:
agent_chain.run(input="what is his employee code?")

In [ ]:
from langchain.chat_models import ChatOpenAI
from langchain.chains.conversation.memory import ConversationBufferWindowMemory
from langchain.chains import RetrievalQA


# chat completion llm
llm = ChatOpenAI(
    openai_api_key= "",
    model_name='gpt-3.5-turbo',
    temperature=0.0
)
# conversational memory
conversational_memory = ConversationBufferWindowMemory(
    memory_key='chat_history',
    k=5,
    return_messages=True
)
# retrieval qa chain
qa = RetrievalQA.from_chain_type(llm=ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0, max_tokens=3000), chain_type="stuff", retriever=retriever)
qa.run(query)

"Dylan's annual salary is $231,141."

In [ ]:
conversational_memory

ConversationBufferWindowMemory(chat_memory=ChatMessageHistory(messages=[]), output_key=None, input_key=None, return_messages=True, human_prefix='Human', ai_prefix='AI', memory_key='chat_history', k=5)

In [ ]:
from langchain.agents import Tool, initialize_agent
from langchain import LLMMathChain
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
tools = [
    Tool(
        name='Knowledge Base',
        func=qa.run,
        description=(
            'use this tool when answering general knowledge queries to get '
            'more information about the topic'
        )
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
]

In [ ]:
from langchain.agents import AgentType
agent = initialize_agent(
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)


In [ ]:
from langchain.agents import initialize_agent

agent = initialize_agent(
    agent='chat-conversational-react-description',
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)

In [ ]:
agent(query)



> Entering new  chain...


InvalidRequestError: ignored

In [ ]:
llm = ChatOpenAI(temperature=0, model="gpt-3.5-turbo-0613")
search = SerpAPIWrapper()
llm_math_chain = LLMMathChain.from_llm(llm=llm, verbose=True)
db = SQLDatabase.from_uri("sqlite:///../../../../../notebooks/Chinook.db")
db_chain = SQLDatabaseChain.from_llm(llm, db, verbose=True)
tools = [
    Tool(
        name = "Search",
        func=search.run,
        description="useful for when you need to answer questions about current events. You should ask targeted questions"
    ),
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math"
    ),
    Tool(
        name="FooBar-DB",
        func=db_chain.run,
        description="useful for when you need to answer questions about FooBar. Input should be in the form of a question containing full context"
    )
]

In [ ]:

from langchain.agents import Too, initialize_agent
from langchain import LLMMathChain
llm_math_chain = LLMMathChain(llm=llm)
tools = [
    Tool(
        name="Calculator",
        func=llm_math_chain.run,
        description="useful for when you need to answer questions about math",
        return_direct=True
    )
]

In [ ]:
from langchain.agents import AgentType
agent = initialize_agent(
    agent=AgentType.OPENAI_FUNCTIONS,
    tools=tools,
    llm=llm,
    verbose=True,
    max_iterations=3,
    early_stopping_method='generate',
    memory=conversational_memory
)



In [ ]:
import openai
def generate_response(query):
    response = openai.ChatCompletion.create(
  engine="test1",
  messages = [{"role":"system","content":"You are an AI assistant that helps people find information based on the information stored in docs. Try your best to answer the question.Firstly greet the user. Limit tokens to 2000."},{"role": "user", "content": query}],
  temperature=0,
  max_tokens=800,
  top_p=1,
  frequency_penalty=0,
  presence_penalty=0,
  stop=None)

    # Return the generated response text
    return response.choices[0].message.content

response = generate_response(query)
print(response)

Hello! How can I assist you today? Regarding your question, I would need more information about what kind of employee data you are looking for on Dylan. Is there a specific company or organization you are referring to? Additionally, please note that I am limited to processing up to 2000 tokens of information at a time.


In [ ]:
import openai
from langchain.retrievers import AzureCognitiveSearchRetriever

def chat_with_ai():
    # Define the chat history list
    chat_history = []
    chat_history.clear()

    # Create the retriever
    retriever = AzureCognitiveSearchRetriever(content_key="content")

    # Add the initial system message to the chat history
    #chat_history.append({"role": "system", "content": "You are an AI assistant that helps people find information based on the information stored. Try your best to answer the question. Firstly greet the user. Limit tokens to 2000."})

    # Start the conversation loop
    while True:
        # Add the greeting message to the chat history
        chat_history.append({"role": "system", "content": "You are an AI assistant that helps people find information based on the information stored. Try your best to answer the question. Firstly greet the user. Limit tokens to 2000."})
        # Get user input
        query = input("User: ")

        # Check if the user wants to exit
        if query.lower() == "exit":
            break

        # Add user query to the chat history
        chat_history.append({"role": "user", "content": query})

        # Make the retrieval request
        documents = retriever.get_relevant_documents(query)

        # Extract relevant information from the retrieved documents
        # and add it to the chat history
        for doc in documents:
            content = doc.page_content
            chat_history.append({"role": "assistant", "content": content})



        # Make the chat API request
        response = openai.ChatCompletion.create(
            engine="test1",
            messages=chat_history,
            temperature=0,
            max_tokens=800,
            top_p=1,
            frequency_penalty=0,
            presence_penalty=0,
            stop=None,
            )

        # Get the AI's response
        answer = response.choices[0].message.content

        # Add AI's response to the chat history
        chat_history.append({"role": "assistant", "content": answer})

        # Print the AI's response
        print("AI: " + answer)

# Call the chat_with_ai function to start the conversation
chat_with_ai()

